# News Article Classification using PyTorch

A text classification model to automatically categorize news articles into four classes: World, Sports, Business, and Sci/Tech.

## Installation

In [ ]:
%%time
%pip install pandas numpy==1.26.4 seaborn==0.9.0 matplotlib scikit-learn portalocker>=2.0.0 plotly
%pip install torch==2.3.0+cpu torchdata==0.9.0+cpu torchtext==0.18.0+cpu \
    --index-url https://download.pytorch.org/whl/cpu

## Import Libraries

In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torchtext.data.utils import get_tokenizer

import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from torchtext.datasets import AG_NEWS
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
from sklearn.manifold import TSNE
import plotly.graph_objs as go

import warnings
warnings.filterwarnings('ignore')

## Helper Functions

In [ ]:
def plot(COST, ACC):
    """Plot training loss and accuracy"""
    fig, ax1 = plt.subplots()
    color = 'tab:red'
    ax1.plot(COST, color=color)
    ax1.set_xlabel('epoch', color=color)
    ax1.set_ylabel('total loss', color=color)
    ax1.tick_params(axis='y', color=color)
    
    ax2 = ax1.twinx()
    color = 'tab:blue'
    ax2.set_ylabel('accuracy', color=color)
    ax2.plot(ACC, color=color)
    ax2.tick_params(axis='y', color=color)
    fig.tight_layout()
    plt.show()

## Load and Prepare Dataset

In [ ]:
# Load AG_NEWS dataset
train_iter = iter(AG_NEWS(split="train"))
y, text = next(train_iter)
print(f"Sample: {y}, {text}")

# Label mapping
ag_news_label = {1: "World", 2: "Sports", 3: "Business", 4: "Sci/Tec"}
print(f"Label: {ag_news_label[y]}")

# Get number of classes
num_class = len(set([label for (label, text) in train_iter]))
print(f"Number of classes: {num_class}")

## Build Vocabulary

In [ ]:
# Reinitialize train_iter
train_iter = AG_NEWS(split="train")

# Define tokenizer
tokenizer = get_tokenizer("basic_english")

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text.lower())

# Build vocabulary
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

print(f"Vocabulary size: {len(vocab)}")
print(f"Sample token indices: {vocab(['age', 'hello'])}")

## Create Train/Validation/Test Split

In [ ]:
# Split dataset
train_iter, test_iter = AG_NEWS()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

# Create 95/5 train/validation split
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Data Processing Pipeline

In [ ]:
def text_pipeline(x):
    return vocab(tokenizer(x))

def label_pipeline(x):
    return int(x) - 1

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for _label, _text in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

## Create DataLoaders

In [ ]:
BATCH_SIZE = 64

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)

# Verify data
label, text, offsets = next(iter(valid_dataloader))
print(f"Batch shapes - Labels: {label.shape}, Text: {text.shape}, Offsets: {offsets.shape}")

## Define Model

In [ ]:
class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

## Initialize Model

In [ ]:
emsize = 64
vocab_size = len(vocab)

model = TextClassificationModel(vocab_size, emsize, num_class).to(device)
print(model)

# Verify output shape
predicted_label = model(text, offsets)
print(f"Output shape: {predicted_label.shape}")

## Prediction and Evaluation Functions

In [ ]:
def predict(text, text_pipeline):
    """Predict the category of a text article"""
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return ag_news_label[output.argmax(1).item() + 1]

def evaluate(dataloader):
    """Evaluate model accuracy on a dataset"""
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc / total_count

# Test before training
print(f"Accuracy before training: {evaluate(test_dataloader):.4f}")
print(f"Sample prediction: {predict('I like sports', text_pipeline)}")

## Training Setup

In [ ]:
LR = 0.1
EPOCHS = 10

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

## Train Model

In [ ]:
cum_loss_list = []
acc_epoch = []
acc_old = 0

for epoch in tqdm(range(1, EPOCHS + 1)):
    model.train()
    cum_loss = 0
    
    for idx, (label, text, offsets) in enumerate(train_dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        cum_loss += loss.item()

    cum_loss_list.append(cum_loss)
    accu_val = evaluate(valid_dataloader)
    acc_epoch.append(accu_val)
    
    print(f"Epoch {epoch}: Loss = {cum_loss:.4f}, Validation Accuracy = {accu_val:.4f}")

    if accu_val > acc_old:
        acc_old = accu_val
        torch.save(model.state_dict(), 'best_model.pth')
        print(f"  -> Model saved with accuracy: {accu_val:.4f}")

## Plot Training Results

In [ ]:
plot(cum_loss_list, acc_epoch)

## Evaluate on Test Set

In [ ]:
test_accuracy = evaluate(test_dataloader)
print(f"Test Accuracy: {test_accuracy:.4f}")

## Visualize Embeddings (t-SNE)

In [ ]:
# Get embeddings from validation batch
batch = next(iter(valid_dataloader))
label, text, offsets = batch
text = text.to(device)
offsets = offsets.to(device)

embedded = model.embedding(text, offsets)
embeddings_numpy = embedded.detach().cpu().numpy()

# Perform t-SNE
X_embedded_3d = TSNE(n_components=3).fit_transform(embeddings_numpy)

# Create 3D scatter plot
trace = go.Scatter3d(
    x=X_embedded_3d[:, 0],
    y=X_embedded_3d[:, 1],
    z=X_embedded_3d[:, 2],
    mode='markers',
    marker=dict(
        size=5,
        color=label.numpy(),
        colorscale='Viridis',
        opacity=0.8
    )
)

layout = go.Layout(
    title="3D t-SNE Visualization of Embeddings",
    scene=dict(
        xaxis_title='Dimension 1',
        yaxis_title='Dimension 2',
        zaxis_title='Dimension 3'
    )
)

fig = go.Figure(data=[trace], layout=layout)
fig.show()

## Test with Sample Article

In [ ]:
article = """Canada navigated a stiff test against the Republic of Ireland on a rain soaked evening in Perth, 
coming from behind to claim a vital 2-1 victory at the Women's World Cup. Katie McCabe opened the scoring 
with an incredible Olimpico goal – scoring straight from a corner kick – as her corner flew straight over 
the despairing Canada goalkeeper Kailen Sheridan at Perth Rectangular Stadium in Australia."""

result = predict(article, text_pipeline)
print(f"Article: {article}\n")
print(f"Predicted Category: {result}")

## Load Saved Model (Optional)

In [ ]:
# To load the best saved model
model.load_state_dict(torch.load('best_model.pth'))
model.eval()
print("Model loaded successfully!")

## Batch Prediction on Multiple Articles

In [ ]:
new_articles = [
    "International talks have made significant headway with the signing of a climate accord.",
    "In a stunning upset, the underdog team won the national title.",
    "Market analysts are optimistic as the tech startup's stock prices soared.",
    "A recent study suggests that a new drug has shown promise in treating Alzheimer's disease.",
    "The sports world is in shock as a legendary player announces their retirement."
]

for i, article in enumerate(new_articles, start=1):
    prediction = predict(article, text_pipeline)
    print(f"Article {i}: {prediction}")
    print(f"  Text: {article[:80]}...\n")